Initial workflow through for CNVnator.  
- Started 8/2/16
- Amanda Everitt
- cwd= /home/everitt/CNV/CNVnator

In [ ]:
#RUN CNVnator
%%bash
#for i in `ls *.bam`
for i in `ls CVG*`
    do
    echo $i
    /home/travc/bin/run_cnvnator.sh -root cyp.root -tree $i
    /home/travc/bin/run_cnvnator.sh -root cyp.root -his 500 -d colony_CNV/   #histogram bin size 500
    /home/travc/bin/run_cnvnator.sh -root cyp.root -stat 500                #statistics(pval)  bin size 500
    /home/travc/bin/run_cnvnator.sh -root cyp.root -partition 500           #partitions the chromosomes/scaffolds into long regions bin size 500
    /home/travc/bin/run_cnvnator.sh -root cyp.root -call 500 > $i.cnv_output      #finds potential CNVs bin size 500
    done

In [15]:
#Keeps calls which are 1.mapped 2. significant 3. larger than 1KB
files = ['CMR4MOPTIA0001_realigned.bam.cnv_output','CVGLCYP1A0001_realigned.bam.cnv_output']
for i in files:
    with open(i,'r') as f:
        with open ('{}.filtered'.format(i),'w') as new:
            with open('{}.filtered.bed'.format(i),'w') as bed:
                for line in f:
                    parts= line.split()
                    if 'UNKN' not in parts[1] or 'Y_unplaced' not in parts[1]: #if mapped
                        if float(parts[3]) < 0.05: # if evalue is less than 0.05 (https://www.biostars.org/p/194936/)
                            pos = parts[1].split(':')
                            coordinates= (pos[1]).split('-')
                            bp = int(coordinates[1])- int(coordinates[0])
                            if bp >= 1000:
                                new.write(line)
                                bed.write(str((pos[0][-2:])) + '\t' + coordinates[0] + '\t' + coordinates[1] + '\n')

In [63]:
#Find intersection
%%bash
/home/everitt/bin/bedtools2/bin/intersectBed -r -f 0.5 -a CMR4MOPTIA0001_realigned.bam.cnv_output.filtered.bed -b CVGLCYP1A0001_realigned.bam.cnv_output.filtered.bed > similarities

In [17]:
%%bash
wc -l CMR4MOPTIA0001_realigned.bam.cnv_output.filtered.bed
wc -l CVGLCYP1A0001_realigned.bam.cnv_output.filtered.bed
wc -l similarities # similarites between mopti and cyp
head similarities

242 similarities
379 CMR4MOPTIA0001_realigned.bam.cnv_output.filtered.bed
401 CVGLCYP1A0001_realigned.bam.cnv_output.filtered.bed
2L	21001	24000
2L	27501	29500
2L	250501	254000
2L	260501	267500
2L	283501	285500
2L	373001	383000
2L	551501	554000
2L	603501	606500
2L	837001	847000
2L	960001	967500
